# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [97]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

import os
from dotenv import load_dotenv
# Load .env enviroment variables into the notebook
load_dotenv()
# Get the API key from the environment variable and store as Python variable
gkey = os.getenv("gkey")


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [98]:
file = pd.read_csv('../output_data/city.csv')
file.dropna()
file.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,Ushuaia,75,AR,1630093926,52,-54.8000,-68.3000,42.42,16.11
1,1,Nosy Varika,43,MG,1630093926,85,-20.5833,48.5333,66.36,3.49
2,2,Broken Hill,2,AU,1630093927,52,-31.9500,141.4333,42.51,10.36
3,3,Torbay,90,CA,1630093927,91,47.6666,-52.7314,66.27,24.16
4,4,Chifeng,0,CN,1630093844,71,42.2683,118.9636,55.78,8.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [99]:
gmaps.configure(api_key = gkey)

In [100]:
locations = file[['Latitude', 'Longitude']]
humidity = file['Humidity'].astype(float)

In [101]:
fig = gmaps.figure(center = [31.9686, -99.9018], zoom_level = 2)

In [102]:
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [103]:
weather = file.loc[(file['Max Temp'] >= 58) & (file['Max Temp'] >= 77)]
wind = weather.loc[(weather['Wind Speed'] <= 7)]
ideal = wind.loc[(wind['Cloudiness'] <= 25)]
len(ideal)

40

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [104]:
hotel_df = ideal.reset_index(drop = True)
hotel_df['Hotel Name'] = ""

#param to update
parameters = {
    'radius': 1500,
    'types': 'hotels',
    'keyword':'hotels',
    'key': gkey
}

In [105]:
for index, row in hotel_df.iterrows():
    Latitude = row['Latitude']
    Longitude = row['Longitude']
    parameters['location'] = f'{Latitude} , {Longitude}'
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    hotel_name = requests.get(base_url, params = parameters)
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent = 4, sort_keys = True))
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel_name['results'][0]['name']
    except (KeyError, IndexError):
        print('....Missing Data.....Skippping.......')

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
....Missing Data.....Skippping.......
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
....Missing Data.....Skippping.......
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 42.2061168,
                    "lng": -121.7441983
                },
                "viewport": {
                    "northeast": {
                        "lat": 42.20766377989273,
                        "lng": -121.7432067201073
                    },
                    "southwest": {
                        "lat": 42.20496412010728,
                        "lng": -121.7459063798927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 32.679527,
                    "lng": -17.104254
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.68091652989272,
                        "lng": -17.10285772010727
                    },
                    "southwest": {
                        "lat": 32.67821687010727,
                        "lng": -17.10555737989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Hotel Enotel Ba\u00eda",
            "opening_hours": {
                "open_now": true
        

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
....Missing Data.....Skippping.......
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
....Missing Data.....Skippping.......
{
    "html_attributions": [],
    "next_page_token": "Aap_uEDM5TtHYQSyJxx4fVtuPWXGp6WOPjw4pkhxNh2h2TXT_LNkMma-JJmD5YWXrrHU5MwY0gK6BsR1Fkx1hIFO4kPsdzwHYtl2DF3NahyDey5Z_OCNXnEFAP15EW60ysv7dWG4OMBd2rVcsR5J08Gcr4EUmk8gPRC5XILYP0M4NTVeQKMt6GgzwepbJynznIC313DDOz3YrZZLYFMYfBg-RkkKKGwVc1UmO3v2ggL5UwP4rwJBDtr8anNdNMC6j34tHOT5Jc0cJltshScverZ2HzyykB1U2IMmYTu1i_GCSYHmCu32WyXLeYbDmZ_p51ToRlH9YStqopkCm_fZmXhI6gH0sPTdKZmRc70KfibuUAzt1fKKyeq8dBWVOvjJD3N3V5B_c39IJUNdfU7t4jOqSO1WYx7iFV5raNyeCtD5wBfc_YKbJ5garSXKMMX1pbflxA",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 5.4231701,
                    "lng": 100.3357086
                },
                "viewport

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
....Missing Data.....Skippping.......
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
....Missing Data.....Skippping.......
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 33.7227281,
                    "lng": 135.9915719
                },
                "viewport": {
                    "northeast": {
                        "lat": 33.72407617989273,
                        "lng": 135.9930596298927
                    },
                    "southwest": {
                        "lat": 33.72137652010728,
                        "lng": 135.9903599701073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_co

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 37.4842965,
                    "lng": -4.4566024
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.48571262989272,
                        "lng": -4.455640470107278
                    },
                    "southwest": {
                        "lat": 37.48301297010727,
                        "lng": -4.458340129892722
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "MS Fuente Las Piedras",
            "opening_hours": {
                "open_now": true
        

...

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
....Missing Data.....Skippping.......
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 14.8854111,
                    "lng": -24.497278
                },
                "viewport": {
                    "northeast": {
                        "lat": 14.88662957989272,
                        "lng": -24.49587112010727
                    },
                    "southwest": {
                        "lat": 14.88392992010728,
                        "lng": -24.49857077989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet"

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 4.938564599999999,
                    "lng": -52.3324381
                },
                "viewport": {
                    "northeast": {
                        "lat": 4.939965729892721,
                        "lng": -52.33110107010727
                    },
                    "southwest": {
                        "lat": 4.937266070107277,
                        "lng": -52.33380072989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Hotel Amazonia Cayenne",
            "opening_hours": {
                "open_now": true

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
....Missing Data.....Skippping.......
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
....Missing Data.....Skippping.......
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
....Missing Data.....Skippping.......
{
    "html_attributions": [],
    "next_page_token": "Aap_uEAIxezG-dwtGBeknv7ujSh5XkcaMMx20Sdm4vZxio9-4h__Vb7070_wJ1QKOstrABoEEfn1khpoyigVn_nXMdOZ9SsinbWGuR0NyqKIhr5yfj3hD-TGR7Sv3Y4__cNZ_ZO8q9rRljR3xFfWQdcUtoa34NS4f4Qtn_czmBxHsVGcDunWWDM1_sEPH7rrl6dnZlgNyA1tO1ItlVu_O_8prnu9xFhSEEUfghsThZ4KdOxxLqMmvdQBT_2f-6-bxnLiZHQeeOMwYbUijF--NRMEmXCjWC1spvotxc8d049MezoqguO0aNtBGWuzsgz2JDjZ8-uiZXXMGNrQnO3nks3-DT7ye-lplmlEgcjhK-5YXIChPbfBIXiuzmf9wOurNCK2U1bYEXYvccYhf4oDpmciolZoserwiNcTVZQA62mNJfhhe-s6ewj3eYTBGmAPGmGkRQ",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
  

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 35.4867059,
                    "lng": -86.44663539999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 35.48841872989272,
                        "lng": -86.44523707010727
                    },
                    "southwest": {
                        "lat": 35.48571907010728,
                        "lng": -86.44793672989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Best Western Shelbyville Inn And Suites Celebration Inn",
            "opening_hours": {

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
....Missing Data.....Skippping.......
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 27.441217,
                    "lng": 120.62376
                },
                "viewport": {
                    "northeast": {
                        "lat": 27.44230932989272,
                        "lng": 120.6251620298927
                    },
                    "southwest": {
                        "lat": 27.43960967010728,
                        "lng": 120.6224623701072
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
  

In [106]:
 hotel_df

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,11,Bethel,1,US,1630093929,69,41.3712,-73.4140,92.80,3.29,
1,24,Lincoln,1,US,1630093717,45,40.8000,-96.6670,98.13,3.00,
2,28,Altamont,1,US,1630093935,33,42.2068,-121.7372,77.77,4.00,Days Inn by Wyndham Klamath Falls
3,37,Ponta do Sol,21,PT,1630093939,73,32.6667,-17.1000,77.00,4.74,Hotel Enotel Baía
4,65,Westport,20,US,1630093948,63,41.1415,-73.3579,94.08,3.00,
5,83,Mayo,1,US,1630093954,75,38.8876,-76.5119,95.36,7.00,
6,96,George Town,20,MY,1630093850,91,5.4112,100.3354,78.89,1.01,"Eastern & Oriental Hotel, Penang"
7,113,Half Moon Bay,20,US,1630093963,66,37.4636,-122.4286,100.06,4.61,
8,141,Caceres,2,ES,1630093973,28,39.5220,-6.3748,81.81,6.73,
9,154,Shingū,2,JP,1630093977,84,33.7333,135.9833,77.92,4.70,Shingu UI Hotel


In [108]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [109]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content = [f"Hotel Location: {rate}" for rate in hotel_info])
fig.add_layer(marker)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))